In [21]:
%matplotlib inline
import numpy as np
import pandas as pd
import random
import json
#import math
#import matplotlib.pyplot as plt
#from initialize_model import *
from functions.helpers import organise_data, hypothetical_series, get_specific_bootstraps_moments, confidence_interval
#from functions.inequality import gini
#from model import *
#import statsmodels.api as sm
from functions.stylizedfacts import autocorrelation_returns
#from matplotlib import style
from functions.indirect_calibration import quadratic_loss_function
import scipy.stats as stats
from SALib.sample import latin
from hurst import compute_Hc

# Asset price volatility and wealth inequality

This notebook contains the following steps:

1. Parameter calibration and estimation
2. Model dynamics
3. Experiment

## 1 Parameter calibration and estimation

### 1.1 Collect data

In [2]:
shiller_data = pd.read_excel('http://www.econ.yale.edu/~shiller/data/ie_data.xls', header=7)[:-3]
p = pd.Series(np.array(shiller_data.iloc[1174:-1]['Price']))
price_div = pd.Series(np.array(shiller_data.iloc[1174:-1]['CAPE']))
p_returns = pd.Series(np.array(shiller_data.iloc[1174:]['Price'])).pct_change()[1:]
pd_returns = pd.Series(np.array(shiller_data.iloc[1174:]['CAPE'])).pct_change()[1:]

### 1.2 calibration

First, I set two parameters for computational efficiency

Then, I calibrate parameters using data and literature. 

In [3]:
params = {"trader_sample_size": 10, # selected for comp efficiency
          "n_traders": 1000, # selected for comp efficiency
          "init_stocks": int((21780000000 / 267.33) / float(1000000)), # market valuation of Vanguard S&P 500 / share price 
          "ticks": len(p), # lenght of reference data
          "fundamental_value": p.mean(), # average value of reference data, assuming efficient markets
          "std_fundamental": p_returns.std(), # standard deviation of returns sp price, assuming efficient markets
          "base_risk_aversion": 0.7, # estimate from Kim & Lee (2012)
          'spread_max': 0.004087, # estimate from Riordan & Storkenmaier (2012)
          "horizon": int(len(p) * 0.35), # estimate based on average churn ratio found by Cella, Ellul and Giannetti (2013)
          # estimated parameters
          "std_noise": 0.01, 
          "w_random": 1.0, 
          "strat_share_chartists": 0.0,
          # parameter only used for experiment
          "mean_reversion": 0.0,
          # fixed / not modelled parameters
          "fundamentalist_horizon_multiplier": 1.0,
          "mutation_intensity": 0.0,
          "average_learning_ability": 0.0,
          "trades_per_tick": 1
         }

In [4]:
#params

Finally, there are seven parameters left which are difficult to calibrate. Therefore, I estimate these values using the method of simulated moments. The starting point of this method is finding appropriate moments which the model should be able to replicate. I note that there should be more moments than parameters. Since 3 parameters need to be estimated, 4 moments are needed. Since this is a highly stylized model, I am only interested in the model replicating some basic moments of the price return series. The moments are the autocorrelation, autocorrelation of absolute returns, kurtosis of returns and hurst of the price. 

First, I calculate these moments for the empirical data. 

In [5]:
emp_moments = np.array([
    autocorrelation_returns(p_returns, 25),
    autocorrelation_returns(p_returns.abs(), 25),
    p_returns.kurtosis(),
    compute_Hc(p, kind='price', simplified=True)[0]
    ])
emp_moments
np.save('emp_moments', emp_moments)

To account for the fact that some of these moments might correlate over different Monte Carlo Simulations, the MSM seeks to obtain a variance covariance matrix of the moments. Since there is only one empirical reality, I use a bootstrap procedure to create a covariance matrix of empirical moments. For this, I use a block bootstrap procedure.

In [6]:
BLOCK_SIZE = 25
BOOTSTRAPS = 100

In [7]:
p_data_blocks = []
price_data_blocks = []
for x in range(0, len(p_returns[:-3]), BLOCK_SIZE):
    p_data_blocks.append(p_returns[x:x + BLOCK_SIZE])
    price_data_blocks.append(p[x:x + BLOCK_SIZE])

In [10]:
bootstrapped_p_returns = []
bootstrapped_prices = []
for i in range(BOOTSTRAPS):
    sim_data_p = [random.choice(p_data_blocks) for _ in p_data_blocks]
    sim_data2_p = [j for i in sim_data_p for j in i]
    bootstrapped_p_returns.append(sim_data2_p)
    
    sim_data_price = [random.choice(price_data_blocks) for _ in price_data_blocks]
    sim_data2_price = [j for i in sim_data_price for j in i]
    bootstrapped_prices.append(sim_data2_price)

In [11]:
rets_autocor = []
rets_abs_autocors = []
kurts = []
hursts = []

for rets, prices in list(zip(bootstrapped_p_returns, bootstrapped_prices)):
    rets_autocor.append(autocorrelation_returns(rets, 25))
    rets_abs_autocors.append(autocorrelation_returns(np.abs(rets), 25))
    kurts.append(pd.Series(rets).kurtosis())
    hursts.append(compute_Hc(prices, kind='price', simplified=True)[0])

In [12]:
all_bootstrapped_moments = [
                            rets_autocor,
                            rets_abs_autocors,
                            kurts,
                            hursts
                           ]

In [13]:
av_moments = [np.nanmean(x) for x in all_bootstrapped_moments]
moments_b = [get_specific_bootstraps_moments(all_bootstrapped_moments, n) for n in range(len(bootstrapped_p_returns))]

Here, I follow [Franke & Westerhoff 2016](https://link.springer.com/article/10.1007/s11403-014-0140-6#Sec8) in that I use the inverse of the bootstrap estimate of the moment covariance matrix as my weights.

In [14]:
W_hat = 1.0 / len(bootstrapped_p_returns) * sum([np.dot(np.array([(mb - av_moments)]).transpose(), np.array([(mb - av_moments)])) for mb in moments_b])
W = np.linalg.inv(W_hat)
np.save('distr_weighting_matrix', W)

I establish confidence intervals for the moments

In [15]:
confidence_intervals = [confidence_interval(m, emp) for m, emp in zip(all_bootstrapped_moments, emp_moments)]

In [16]:
j_values = []
for b in moments_b:
    j_values.append(quadratic_loss_function(b, emp_moments, W))

In [17]:
scores = [0 for x in moments_b[0]]
for bootstr in range(len(moments_b)):
    for idx, moment in enumerate(moments_b[bootstr]):
        if moment > confidence_intervals[idx][0] and moment < confidence_intervals[idx][1]:
            scores[idx] += 1
MCR_bootstrapped_moments = np.array(scores) / (np.ones(len(scores)) * len(moments_b))

To start the estimation procedure, I first sample the parameter space using Latin Hypercube sampling

In [18]:
population_size = 1

In [19]:
problem = {
  'num_vars': 3,
  'names': ['std_noise', "w_random", "strat_share_chartists"],
  'bounds': [[0.03, 0.09], [0.02, 0.15], [0.02, 0.7]]
}

In [22]:
latin_hyper_cube = latin.sample(problem=problem, N=population_size)
latin_hyper_cube = latin_hyper_cube.tolist()
with open('hypercube.txt', 'w') as f:
    json.dump(latin_hyper_cube, f)
initial_params = latin_hyper_cube[0]
initial_params

[0.05704321942550724, 0.11590615914660493, 0.502318253496888]

I perform the estimation excercise in a different Python file using multi-processing. These will serve as the starting parameters to do simulation. 